# A Small Interactive LLM Playground
A simple interactive playground to experiment with text generation. It is meant purely for exploration

- Create input widgets for the prompt, model selection, decoding strategy, and temperature
- Use Hugging Face's generate method to produce text based on the selected settings
- Display the model's response directly in the notebook output
 
You may find following links helpful:
- https://ipywidgets.readthedocs.io/en/latest/
- https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In [1]:
import ipywidgets as widgets
from IPython.display import display, Markdown

In [4]:
# Steps to implement:
# 1. Load models and tokenizers (GPT-2 and Qwen).
# 2. Define a helper function to generate text with different decoding strategies.
# 3. Create interactive UI elements (prompt box, model selector, strategy selector, temperature slider).
# 4. Add a button to trigger text generation.
# 5. Define the button’s behavior.
# 6. Display the full UI for the playground.

# Load Models
try:
    tokenizers
    models
except NameError:
    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch
    MODELS = {
        "gpt2": "gpt2",
        "qwen": "Qwen/Qwen3-0.6B" 
    }

    tokenizers, models = {}, {}
    device = "cuda" if torch.cuda.is_available() else "mps"

    for key, mid in MODELS.items():
        tok = AutoTokenizer.from_pretrained(mid)
        mdl = AutoModelForCausalLM.from_pretrained(mid).eval().to(device)
        tokenizers[key], models[key] = tok, mdl
        print(f"Loaded {mid} as {key}")

# generate text
def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    enc = tok(prompt, return_tensors="pt").to(mdl.device)
    gen_args = dict(**enc, max_new_tokens=max_new_tokens)
    if strategy == "greedy":
        gen_args["do_sample"] = False
    elif strategy == "top_k":
        gen_args.update(dict(do_sample=True, top_k=50, temperature=temperature))
    elif strategy == "top_p":
        gen_args.update(dict(do_sample=True, top_p=0.9, temperature=temperature))
    else:
        raise ValueError("Unknown strategy")
    out = mdl.generate(**gen_args)
    return tok.decode(out[0], skip_special_tokens=True)

# Widgets
prompt_box = widgets.Textarea(
    value="Tell me a fun fact about space.",
    placeholder="Type your prompt here",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="120px")
)

model_dropdown = widgets.Dropdown(
    options=[("GPT-2", "gpt2"), ("Qwen3-0.6B", "qwen")],
    value="gpt2",
    description="Model:"
)

strategy_dropdown = widgets.Dropdown(
    options=[("Greedy", "greedy"), ("Top-k", "top_k"), ("Top-p", "top_p")],
    value="greedy",
    description="Strategy:"
)

temperature_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Temp:"
)

generate_button = widgets.Button(description="Generate", button_style="primary")
output_area = widgets.Output()

# On_generate function
def on_generate(_):
    # Change button to loading state
    generate_button.description = "⏳ Generating..."
    generate_button.disabled = True
    generate_button.button_style = "warning"
    
    output_area.clear_output()
    with output_area:
        try:
            result = generate_playground(
                model_dropdown.value,
                prompt_box.value,
                strategy_dropdown.value,
                temperature_slider.value
            )
            display(Markdown(f"**Output:**\n\n{result}"))
        except Exception as e:
            print("Error:", e)
        finally:
            # Reset button to original state
            generate_button.description = "Generate"
            generate_button.disabled = False
            generate_button.button_style = "primary"

generate_button.on_click(on_generate)

# Define UI
ui = widgets.VBox([
    prompt_box,
    widgets.HBox([model_dropdown, strategy_dropdown, temperature_slider]),
    generate_button,
    output_area
])

display(ui)
